<a href="https://colab.research.google.com/github/thisisHJLee/KoBERT-practice/blob/main/KoBERT%EB%A1%9C_%EB%8B%A4%EC%A4%91_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference:\
[[파이썬]KoBERT로 다중 분류 모델 만들기](https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab)\
[Naver Sentiment Analysis Fine-Turning with pytorch](https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb#scrollTo=itIExnuLbSap)\
[깃허브 Chatbot_data](https://github.com/songys/Chatbot_data)

## 1. 라이브러리 설치

In [ ]:
!kill -9 -1

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595752 sha256=8eee6a2c3140183fac767012a5af306138e501389f60ed39e18e870d415b7af4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.9 MB/s 


In [ ]:
!pip uninstall transformers
!pip uninstall torch

!pip install transformers==3
!pip install torch

Found existing installation: torch 1.12.1+cu113
Uninstalling torch-1.12.1+cu113:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.12.1+cu113.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
    /usr/local/lib/python3.7/dist-packages/torchgen/*
Proceed (y/n)? y
y
  Successfully uninstalled torch-1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 754 kB 34.7 MB/s 
     |████████████████████████████████| 880 kB 48.0 MB/s 
     |████████████████████████████████| 3.0 MB 51.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8610d98b29d30923fb668b1d77951db55b7868a81cf1cd9403b7806cef1197f1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eee

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_ocfa4ve
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_ocfa4ve
     |████████████████████████████████| 129 kB 25.2 MB/s 
     |████████████████████████████████| 54.7 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 41.6 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x390a4000 @  0x7f323caa1615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## 2. 데이터 불러오기 & 전처리

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/딥로딩/ChatbotData.csv')

In [ ]:
chatbot_data.sample(n=10)

,Q,A,label
8594,헤어진 연인의 새로운 사랑,이젠 그분을 놓아주세요.,1
697,날씨 풀렸다,따뜻해졌죠.,0
425,꽃 선물 좋아할까,꽃 선물은 언제나 좋죠.,0
7087,"연락왔는데,진짜 미칠거같네",다시 헤어진 그 때로 돌아간 느낌이죠.,1
10103,스킨십 좋아하는 남자친구 어떡해?,서로 속도를 맞춰보세요.,2
5901,너 만든 사람은 누구야?,마음과 마음을 이어보고 싶어하는 사람이 만들었어요.,1
9927,사랑이 변하면 어떡하지?,사랑은 변화하는 거예요. 두려워하지 마세요.,2
7690,이제 여친이 마음이 괜잖아진거같아!!!,다행으로 생각할게요.,1
6299,먼저 헤어지자고 하면 후폭풍 있을까?,후폭풍은 누구에게나 와요.,1
8583,헤어진 여자친구를 다시 만나고 싶어,후회할 것 같다면 늦지 않게 연락해보세요.,1


In [ ]:
data_list = []
for q, label in zip(chatbot_data['Q'], chatbot_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(len(data_list))
print(data_list[0])
print(data_list[600])
print(data_list[1200])
print(data_list[2500])
print(data_list[5000])
print(data_list[10000])
print(data_list[-1])

11823
['12시 땡!', '0']
['나도 월급 필요해', '0']
['대리운전 불렀어', '0']
['셀프웨딩 어떨꺼 같아', '0']
['학원폭력 짜증나', '0']
['사랑한다고 말해주면 뭐가 덧나나', '2']
['힘들어서 결혼할까봐', '2']


## 3. Train data & Test data

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

8867
2956


## 4. KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 3647, 6356, 7873,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 5. KoBERT 학습모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## 6. KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/139 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0750200748443604 train acc 0.40625
epoch 1 train acc 0.5292105601233299


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/47 [00:00<?, ?it/s]

...

epoch 19 test acc 0.8835328014184397


  0%|          | 0/139 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.0007329117506742477 train acc 1.0
epoch 20 train acc 0.9939298561151079


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 20 test acc 0.8832003546099291


## 7. 모델 저장하기

In [ ]:
import os
os.chdir('/content/drive/MyDrive/딥로딩/chatbot/models/')
os.getcwd()

'/content/drive/MyDrive/딥로딩/chatbot/models'

In [ ]:
path = '/content/drive/MyDrive/딥로딩/chatbot/models/'
torch.save(model, path + '3emotions_model.pt')  # 전체 모델 저장

In [ ]:
torch.save(model.state_dict(), '3emotions_model_state_dict.pt')  # 모델 객체의 state_dict 저장

In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '3emotions_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

## 8. 모델 불러오기

In [ ]:
import os
os.chdir('/content/drive/MyDrive/딥로딩/chatbot/models/')

model1 = torch.load('3emotions_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('3emotions_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('3emotions_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

## 9. 테스트

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/content/drive/MyDrive/딥로딩/chatbot/models/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("일상(중립)이")
            elif np.argmax(logits) == 1:
                test_eval.append("이별(부정)이")
            elif np.argmax(logits) == 2:
                test_eval.append("사랑(긍정)이")


        print(test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 반복!
end = 1
while end == 1 :
    sentence = input("말을 입력해주세요 : ")
    if sentence == "exit" :
        break
    predict(sentence)
    print("\n")

말을 입력해주세요 : 안녕
일상(중립)이 느껴집니다.


말을 입력해주세요 : 어제 데이트 즐거웠어
일상(중립)이 느껴집니다.


말을 입력해주세요 : 썸녀랑 카톡했어
사랑(긍정)이 느껴집니다.


말을 입력해주세요 : 썸남이랑 싸웠어ㅠㅠ
사랑(긍정)이 느껴집니다.


말을 입력해주세요 : 친구가 차였대
일상(중립)이 느껴집니다.


말을 입력해주세요 : 어제 헤어졌어ㅜ
이별(부정)이 느껴집니다.


말을 입력해주세요 : 오늘 점심 뭘까?
일상(중립)이 느껴집니다.


말을 입력해주세요 : 걔 너무 짜증나
일상(중립)이 느껴집니다.


말을 입력해주세요 : 친구랑 싸웠어
일상(중립)이 느껴집니다.


말을 입력해주세요 : 남친이랑 싸웠어
사랑(긍정)이 느껴집니다.


말을 입력해주세요 : 여친이랑 싸웠어
사랑(긍정)이 느껴집니다.


말을 입력해주세요 : exit


In [ ]:
#질문 반복!
end = 1
while end == 1 :
    sentence = input("말을 입력해주세요 : ")
    if sentence == "exit" :
        break
    predict(sentence)
    print("\n")

말을 입력해주세요 : 어제 데이트 로맨틱했어


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


사랑(긍정)이 느껴집니다.


말을 입력해주세요 : 꽃 선물 고마워
일상(중립)이 느껴집니다.


말을 입력해주세요 : 손편지 고마워
일상(중립)이 느껴집니다.


말을 입력해주세요 : 여자친구가 귀여워
사랑(긍정)이 느껴집니다.


말을 입력해주세요 : 카톡으로 싸웠어ㅠㅠ
일상(중립)이 느껴집니다.


말을 입력해주세요 : exit
